In [1]:
import os, pickle, sys, torch, numpy as np
from collections import defaultdict
# paths  
# table_path = '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/all_2d1d_datasets/entropy_radius_exps_on_HSQC'

print_keys = [
    "test/mean_rank_1",
    # "test/mean_rank_5",
    # "test/mean_mean_rank",
    "test/mean_cos",
    "test/mean_f1", 
]


In [2]:
exp_name_to_key_to_results = defaultdict(lambda: defaultdict(list))
# for input_type in ['1d', "all_info", "HSQC_and_C", "HSQC_and_H", "only_C", "only_H", "only_HSQC"]:
# for input_type in ['1d']:    
for input_type in ['all_info']:    
  
    # table_path = f'/root/gurusmart/MorganFP_prediction/reproduce_previous_works/db_specific_FP_with_entropy/entropy_radius_exps_{input_type}'
    table_path = '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/fix_combining_dataset_load_mfp_bug/train_on_all_data_possible'
    table_path = '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/puuting_h_in_the_middle/train_on_all_data_possible'
    # table_path = '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/weird_H_and_tautomer_cleaned/train_on_all_data_possible_hsqc_folderset_code'
    table_path = '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/stable_argsort/train_on_all_data_possible'

    all_exps = os.listdir(table_path)
    # load pickles 
    from collections import defaultdict
    for exp in all_exps:
        if 'trial' in exp:
            trial_spelling = "trial"
        elif 'trail' in exp:
            trial_spelling = "trail"
           
        try:     
            # print(exp)
            with open(f'{table_path}/{exp}/test_result.pkl', 'rb') as f:
                result = pickle.load(f)[0]
                
                for key in print_keys:
                    if key not in result:
                        continue
                    exp_name_to_key_to_results["_".join(exp.split(trial_spelling)[:-1])][key].append(result[key])
                    # exp_name_to_key_to_results[exp][key].append(result[key])
                    # print(key, result[key])
            # print(exp_name_to_key_to_results)
        except Exception as e:
            print(f'{table_path}/{exp}/test_result.pkl')            
    
# exp_name_to_key_to_results


In [3]:
# exp_name_to_key_to_results

In [4]:
avg_results = {}
for exp_name, key_to_results in exp_name_to_key_to_results.items():
    avg_results[exp_name] = {key: np.mean(val) for key, val in key_to_results.items()}

In [5]:
# sort based on dict key 
def get_sort_key(item):
    if ((item[0].split("R0_to_R")[-1]).split("_")[0]).isdecimal():
        return int((item[0].split("R0_to_R")[-1]).split("_")[0])
    
    else:
        return float('inf')

# # puttin same modality toghether
# def get_sort_key(x):
#     return ("_".join(x[0].split("_")[3:]))

avg_results = dict(sorted(avg_results.items(),key=get_sort_key ))

In [6]:
# for r in avg_results.items():
#     display(r)
    # print()
avg_results

{'only_hsqc_': {'test/mean_rank_1': np.float64(0.8907606403032938),
  'test/mean_cos': np.float64(0.8381503621737162),
  'test/mean_f1': np.float64(0.8487166166305542)},
 'only_1d_': {'test/mean_rank_1': np.float64(0.956542948881785),
  'test/mean_cos': np.float64(0.9074298540751139),
  'test/mean_f1': np.float64(0.914314885934194)},
 'HSQC_and_C_': {'test/mean_rank_1': np.float64(0.9338107705116272),
  'test/mean_cos': np.float64(0.8539090752601624),
  'test/mean_f1': np.float64(0.8631412386894226)},
 'only_c_': {'test/mean_rank_1': np.float64(0.9457193811734518),
  'test/mean_cos': np.float64(0.8822630445162455),
  'test/mean_f1': np.float64(0.8928653200467428)},
 'HSQC_and_H_': {'test/mean_rank_1': np.float64(0.8607313434282938),
  'test/mean_cos': np.float64(0.8145809570948283),
  'test/mean_f1': np.float64(0.8255233764648438)},
 'only_h_': {'test/mean_rank_1': np.float64(0.7550184528032938),
  'test/mean_cos': np.float64(0.7885106404622396),
  'test/mean_f1': np.float64(0.79664208

In [7]:
# print in latex format
print ("\\begin{table}[h]")
print("\centering")
print('\\begin{tabular}{|c|c|c|c|c|c|}')
print("\hline")
print("Model Input & Rank-1$\\uparrow$ & Rank-5$\\uparrow$ & Mean Rank$\\downarrow$ & Cosine Sim$\\uparrow$ & F1-score$\\uparrow$\\\\")
print("\hline")

for exp_name, key_to_results in avg_results.items():
    name = exp_name
    if name[:2]=="FP":
        name = "R"+name.split(" ")[-1][-2]+" FP"
    print(" ".join(name.split("_")[:3]) , end="")
    for key, val in key_to_results.items():
        if key in ["test/mean_rank_1", "test/mean_rank_5"]:
            print(f' & {val*100:.2f}\%', end="")
        elif key in ["test/mean_mean_rank"]:
            print(f' & {val+1:.2f}', end="")
        else:
            print(f' & {val:.4f}', end="")
    print(" \\\\")
    print("\hline")
    
print('\end{tabular}')
print("\caption{Performance of various MFs when" + f'{exp_name.split("FP")[-1].replace("_", " ")}' + "NMRs are available}")
# print("\caption{which entropy-based FP to use when"+f'{exp_name.split("FP")[-1].replace("_", " ")}NMR' +" is available}")
print('\label{table_name}')
print('\end{table}')

\begin{table}[h]
\centering
\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model Input & Rank-1$\uparrow$ & Rank-5$\uparrow$ & Mean Rank$\downarrow$ & Cosine Sim$\uparrow$ & F1-score$\uparrow$\\
\hline
only hsqc  & 89.08\% & 0.8382 & 0.8487 \\
\hline
only 1d  & 95.65\% & 0.9074 & 0.9143 \\
\hline
HSQC and C & 93.38\% & 0.8539 & 0.8631 \\
\hline
only c  & 94.57\% & 0.8823 & 0.8929 \\
\hline
HSQC and H & 86.07\% & 0.8146 & 0.8255 \\
\hline
only h  & 75.50\% & 0.7885 & 0.7966 \\
\hline
\end{tabular}
\caption{Performance of various MFs whenonly h NMRs are available}
\label{table_name}
\end{table}


In [8]:
exp_name

'only_h_'

not avg here
just looking at multi vs single clsifer

In [9]:
import os, pickle, sys, torch, numpy as np



# paths  
table_path = '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/average_3/model_sizes'

metric = [
    "mean_rank_1",
    # "mean_rank_5",
    # "mean_mean_rank",
    "mean_cos",
    # "mean_f1", 
]

NMRs  = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc" , "only_1d","only_C_NMR", "only_H_NMR"]

all_keys = [ "test_"+m+"_"+nmr for nmr in NMRs for m in metric]


In [10]:
all_keys

['test_mean_rank_1_all_inputs',
 'test_mean_cos_all_inputs',
 'test_mean_rank_1_HSQC_H_NMR',
 'test_mean_cos_HSQC_H_NMR',
 'test_mean_rank_1_HSQC_C_NMR',
 'test_mean_cos_HSQC_C_NMR',
 'test_mean_rank_1_only_hsqc',
 'test_mean_cos_only_hsqc',
 'test_mean_rank_1_only_1d',
 'test_mean_cos_only_1d',
 'test_mean_rank_1_only_C_NMR',
 'test_mean_cos_only_C_NMR',
 'test_mean_rank_1_only_H_NMR',
 'test_mean_cos_only_H_NMR']

In [11]:

# load pickles 
from collections import defaultdict




In [12]:
def show_result(filter_by="multi"):
    
    exp_name_to_key_to_results = defaultdict(lambda: defaultdict(list))
    all_exps = os.listdir(table_path)
    all_exps = list(filter(lambda x: filter_by in x, all_exps))

    for exp in all_exps:
        # print(exp)
        with open(f'{table_path}/{exp}/test_result.pkl', 'rb') as f:
            result = pickle.load(f)[0]
            # print(result)
            # break
            for key in all_keys:
                if key not in result:
                    continue
                exp_name_to_key_to_results["_".join(exp.split("_")[:-1])][key].append(result[key])
                # print(key, result[key])

    avg_results = {}
    for exp_name, key_to_results in exp_name_to_key_to_results.items():
        avg_results[exp_name] = {key: np.mean(val) for key, val in key_to_results.items()}
    avg_results = dict(sorted(avg_results.items()))
    display(avg_results)
    # for exp_name, key_to_results in avg_results.items():
    #     name = exp_name
    
    #     print(f'{name.replace("_", " ")}', end="")
    #     for key, val in key_to_results.items():
    #         if key in ["test/mean_rank_1", "test/mean_rank_5"]:
    #             print(f' & {val*100:.2f}\%', end="")
    #         elif key in ["test/mean_mean_rank"]:
    #             print(f' & {val:.2f}', end="")
    #         else:
    #             print(f' & {val:.4f}', end="")
    #     print(" \\\\")
    #     print("\hline")


show_result()          

FileNotFoundError: [Errno 2] No such file or directory: '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/average_3/model_sizes'

In [ ]:
show_result("flexible_384_t")          

{'flexible_384': {'test_mean_rank_1_all_inputs': 0.9239176114400228,
  'test_mean_cos_all_inputs': 0.849866251150767,
  'test_mean_rank_1_HSQC_H_NMR': 0.8838695685068766,
  'test_mean_cos_HSQC_H_NMR': 0.8238423665364584,
  'test_mean_rank_1_HSQC_C_NMR': 0.9105113744735718,
  'test_mean_cos_HSQC_C_NMR': 0.8393414616584778,
  'test_mean_rank_1_only_hsqc': 0.7970621983210245,
  'test_mean_cos_only_hsqc': 0.7732182741165161,
  'test_mean_rank_1_only_1d': 0.8979419867197672,
  'test_mean_cos_only_1d': 0.8331032395362854,
  'test_mean_rank_1_only_C_NMR': 0.8609496156374613,
  'test_mean_cos_only_C_NMR': 0.8099772930145264,
  'test_mean_rank_1_only_H_NMR': 0.7571683526039124,
  'test_mean_cos_only_H_NMR': 0.7691749930381775}}

In [ ]:
nmr_to_metric_to_value_single = defaultdict(dict)


for i, (k,v) in enumerate(single_dict.items()):
    
    nmr_type = NMR_types[i//14]
    # nmr_type = " ".join(nmr_type.split("_")[2:])
    metric = i%14
    if metric in [4,6, 10, 11, 12]:
        nmr_to_metric_to_value_single[nmr_type][k] = v
    else:
        continue


NameError: name 'single_dict' is not defined

In [ ]:
# nmr_to_metric_to_value_multi = defaultdict(dict)


# for i, (k,v) in enumerate(multi_dict.items()):
    
#     nmr_type = NMR_types[i//14]
#     nmr_type = " ".join(nmr_type.split("_")[2:])
#     metric = i%14
#     if metric in [4,6, 10, 11, 12]:
#         nmr_to_metric_to_value_multi[nmr_type][k] = v
#     else:
#         continue


In [ ]:
# nmr_to_metric_to_value_single

defaultdict(dict,
            {'all_inputs': {'test_mean_cos_all_inputs': 0.8479413390159607,
              'test_mean_f1_all_inputs': 0.8588761687278748,
              'test_mean_mean_rank_all_inputs': 2.579009771347046,
              'test_mean_rank_1_all_inputs': 0.5099216103553772,
              'test_mean_rank_5_all_inputs': 0.9658825397491455},
             'HSQC_H_NMR': {'test_mean_cos_HSQC_H_NMR': 0.8220083713531494,
              'test_mean_f1_HSQC_H_NMR': 0.8360926508903503,
              'test_mean_mean_rank_HSQC_H_NMR': 7.510948657989502,
              'test_mean_rank_1_HSQC_H_NMR': 0.482708215713501,
              'test_mean_rank_5_HSQC_H_NMR': 0.9477534294128418},
             'HSQC_C_NMR': {'test_mean_cos_HSQC_C_NMR': 0.837510883808136,
              'test_mean_f1_HSQC_C_NMR': 0.8496221899986267,
              'test_mean_mean_rank_HSQC_C_NMR': 3.8844082355499268,
              'test_mean_rank_1_HSQC_C_NMR': 0.5028753876686096,
              'test_mean_rank_5_HSQC_C_NMR':

In [ ]:
# latex table
for k,v in nmr_to_metric_to_value_single.items():
    print(k.replace("_", " ")+ " single classifier")
    for print_k in print_keys:
        for metric, val in v.items():
            if print_k in metric:
                if print_k in ["mean_rank_1", "mean_rank_5"]:
                    print(f' & {val*100:.2f}\%', end="")
                elif print_k in ["mean_mean_rank"]:
                    print(f' & {val:.2f}', end="")
                else:
                    print(f' & {val:.4f}', end="")
    print("\\hline")

    print(k.replace("_", " ")+" multi classifier")
    v_multi = nmr_to_metric_to_value_multi[k]
    for print_k in print_keys:                
        for metric, val in v_multi.items():
            if print_k in metric:
                if print_k in ["mean_rank_1", "mean_rank_5"]:
                    print(f' & {val*100:.2f}\%', end="")
                elif print_k in ["mean_mean_rank"]:
                    print(f' & {val:.2f}', end="")
                else:
                    print(f' & {val:.4f}', end="")
    print("\\hline")
   
    

all inputs single classifier
 & 50.99\% & 96.59\% & 2.58 & 0.8479 & 0.8589\hline
all inputs multi classifier
\hline
HSQC H NMR single classifier
 & 48.27\% & 94.78\% & 7.51 & 0.8220 & 0.8361\hline
HSQC H NMR multi classifier
\hline
HSQC C NMR single classifier
 & 50.29\% & 96.48\% & 3.88 & 0.8375 & 0.8496\hline
HSQC C NMR multi classifier
\hline
only hsqc single classifier
 & 44.24\% & 89.46\% & 16.04 & 0.7785 & 0.7949\hline
only hsqc multi classifier
\hline
only 1d single classifier
 & 48.54\% & 94.75\% & 6.19 & 0.8278 & 0.8409\hline
only 1d multi classifier
\hline
only H NMR single classifier
 & 37.68\% & 84.94\% & 36.97 & 0.7602 & 0.7784\hline
only H NMR multi classifier
\hline
only C NMR single classifier
 & 46.34\% & 92.17\% & 14.63 & 0.8033 & 0.8191\hline
only C NMR multi classifier
\hline


In [ ]:
nmr_to_metric_to_value_multi

defaultdict(dict,
            {'': {'test_mean_cos_all_inputs': 0.8487057685852051,
              'test_mean_f1_all_inputs': 0.858889639377594,
              'test_mean_mean_rank_all_inputs': 2.728726863861084,
              'test_mean_rank_1_all_inputs': 0.5132314562797546,
              'test_mean_rank_5_all_inputs': 0.9662854075431824,
              'test_mean_cos_only_hsqc': 0.7571645975112915,
              'test_mean_f1_only_hsqc': 0.7718337774276733,
              'test_mean_mean_rank_only_hsqc': 16.730077743530273,
              'test_mean_rank_1_only_hsqc': 0.4196631610393524,
              'test_mean_rank_5_only_hsqc': 0.8786495327949524,
              'test_mean_cos_only_1d': 0.8341397047042847,
              'test_mean_f1_only_1d': 0.8465748429298401,
              'test_mean_mean_rank_only_1d': 5.494770526885986,
              'test_mean_rank_1_only_1d': 0.4977881610393524,
              'test_mean_rank_5_only_1d': 0.9539307355880737},
             'NMR': {'test_mean_cos_H

In [ ]:
import pickle
for split in ['train','val','test']:
    path = f"/root/gurusmart/MorganFP_prediction/reproduce_previous_works/smart4.5/datasets/{split}_indices_of_full_info_NMRs.pkl"
    indices = pickle.load(open(path, 'rb'))
    print(split, len(indices))

train 31990
val 3986
test 3930
